# Grid Search 실습

### 데이터셋 다운로드

In [11]:
! wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z' -O pima-indians-diabetes.csv #https://drive.google.com/file/d/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z/view?usp=sharing

--2021-05-28 09:53:44--  https://docs.google.com/uc?export=download&id=1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z
Resolving docs.google.com (docs.google.com)... 173.194.194.113, 173.194.194.138, 173.194.194.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.194.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ae10pjnug0ed9q17ob4ah2pngtlea2vk/1622195550000/04591858293269658552/*/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z?e=download [following]
--2021-05-28 09:53:44--  https://doc-0o-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ae10pjnug0ed9q17ob4ah2pngtlea2vk/1622195550000/04591858293269658552/*/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z?e=download
Resolving doc-0o-1s-docs.googleusercontent.com (doc-0o-1s-docs.googleusercontent.com)... 108.177.111.132, 2607:f8b0:4001:c07::84
Connecting to doc-0o-1s-docs.googleusercontent.com (doc-0o

## batch size와 epoch 최적화

In [12]:
#@title
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.697917 using {'batch_size': 10, 'epochs': 100}
0.647135 (0.022628) with: {'batch_size': 10, 'epochs': 10}
0.648438 (0.019918) with: {'batch_size': 10, 'epochs': 50}
0.697917 (0.022628) with: {'batch_size': 10, 'epochs': 100}
0.613281 (0.073079) with: {'batch_size': 20, 'epochs': 10}
0.678385 (0.009744) with: {'batch_size': 20, 'epochs': 50}
0.675781 (0.005524) with: {'batch_size': 20, 'epochs': 100}
0.502604 (0.083800) with: {'batch_size': 40, 'epochs': 10}
0.613281 (0.005524) with: {'batch_size': 40, 'epochs': 50}
0.647135 (0.035849) with: {'batch_size': 40, 'epochs': 100}
0.473958 (0.080771) with: {'batch_size': 60, 'epochs': 10}
0.640625 (0.019918) with: {'batch_size': 60, 'epochs': 50}
0.645833 (0.035132) with: {'batch_size': 60, 'epochs': 100}
0.553385 (0.082700) with: {'batch_size': 80, 'epochs': 10}
0.602865 (0.054718) with: {'batch_size': 80, 'epochs': 50}
0.660156 (0.017758) with: {'batch_size': 80, 'epochs': 100}
0.557292 (0.142005) with: {'batch_size': 100, 'epochs':

## 학습률과 모멘텀 최적화

In [13]:
#@title
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
import time
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	modelx = Sequential()
	modelx.add(Dense(12, input_dim=8, activation='relu'))
	modelx.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	modelx.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return modelx
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
modelx = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=modelx, param_grid=param_grid, n_jobs=-1, cv=3)

# 트레이닝 시작
start = time.time()
print("트레이닝 시작")
grid_result = grid.fit(X, Y)
end = time.time()
print(end - start, ' 초 결렸습니다')    

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

트레이닝 시작


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


450.945716381073  초 결렸습니다
Best: 0.690104 using {'learn_rate': 0.001, 'momentum': 0.4}
0.648438 (0.011049) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.666667 (0.011201) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.690104 (0.010253) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.644531 (0.041463) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.675781 (0.028348) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.657552 (0.028587) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.662760 (0.003683) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.664062 (0.041707) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.649740 (0.026557) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.649740 (0.026557) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.651042 (0.02

In [14]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
import time
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	modelx = Sequential()
	modelx.add(Dense(12, input_dim=8, activation='relu'))
	modelx.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	modelx.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return modelx
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
modelx = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
# learn_rate와 momentum 두 개의 하이퍼파라미터의 그리드를 코드로 정의하십시오. 
# learn_rate는 0.001, 0.01. 0.1, 0.2, 0.3로 하고 momentum은 0.0, 0.2, 0.4, 0.6, 0.8, 0.9로 하십시오.
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=modelx, param_grid=param_grid, n_jobs=-1, cv=3)

# 트레이닝 시작
start = time.time()
print("트레이닝 시작")
grid_result = grid.fit(X, Y)
end = time.time()
print(end - start, ' 초 결렸습니다') 

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

트레이닝 시작


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


440.96605825424194  초 결렸습니다
Best: 0.686198 using {'learn_rate': 0.001, 'momentum': 0.6}
0.632812 (0.047628) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.674479 (0.032106) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.680990 (0.040386) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.686198 (0.008027) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.678385 (0.016367) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.649740 (0.023510) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.661458 (0.032734) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.658854 (0.036690) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.658854 (0.008027) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.648438 (0.028348) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.651042 (0.027126) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.651042 (0.

In [15]:
from keras.layers import Dropout 
layer = Dropout(rate=0.2, input_shape=(2,))

In [18]:
from keras.layers import LeakyReLU
layer = LeakyReLU(alpha=0.1)